In [1]:
# Import libraries 
import pandas as pd
import numpy as np
import json
import re
import time
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import time

In [3]:
# Load dataset
data = pd.read_json('/home/brandon-ism/Documents/Sports_and_Outdoors/Sports_and_Outdoors.json', lines=True)
data.head()

,_id,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,category,class
0,{'$oid': '5a132768741a2384e847b8ed'},A2PAVURT4NOHE1,0000031852,Leah,"[0, 0]",Bought it for a ballet tutu but it is being wo...,5,Super cute,1388361600,"12 30, 2013",Sports_and_Outdoors,1
1,{'$oid': '5a132768741a2384e847b8e9'},A1SNLWGLFXD70K,0000031852,DEVA,"[0, 0]",I origonally didn't get the item I ordered. W...,4,Happy with purchase even though it came a lot ...,1392940800,"02 21, 2014",Sports_and_Outdoors,1
2,{'$oid': '5a132768741a2384e847b8ee'},A3URQ0LXLV46E9,0000031852,shortyvee,"[0, 0]",My daughter and her friends love the colors an...,4,zebralisous,1400544000,"05 20, 2014",Sports_and_Outdoors,1
3,{'$oid': '5a132768741a2384e847b8ea'},A1KJ4CVG87QW09,0000031852,Donna Carter-Scott,"[0, 0]","Arrived very timely, cute grandbaby loves it. ...",4,Cute Tutu,1389657600,"01 14, 2014",Sports_and_Outdoors,1
4,{'$oid': '5a132768741a2384e847b8eb'},AA9ITO6ZLZW6,0000031852,Jazzy77,"[0, 0]",My little girl just loves to wear this tutu be...,5,Versatile,1399507200,"05 8, 2014",Sports_and_Outdoors,1


Here, the input features will be: `reviewText`, `overall`, `summary`, and `helpful`
The predictor will be `class`, which indicates whether the review is spam (1), or not spam (0)

The first element of the `helpful` feature is extracted, indicating the number of users that found that review helpful.

In [4]:
# Extract the relevant columns
data = data[['reviewText', 'overall', 'summary', 'helpful', 'class']]

# Clean the 'helpful' column: extract the first element of the list - num of helpful votes
data['helpful'] = data['helpful'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else 0)

# Check cleaned data
data.head()

,reviewText,overall,summary,helpful,class
0,Bought it for a ballet tutu but it is being wo...,5,Super cute,0,1
1,I origonally didn't get the item I ordered. W...,4,Happy with purchase even though it came a lot ...,0,1
2,My daughter and her friends love the colors an...,4,zebralisous,0,1
3,"Arrived very timely, cute grandbaby loves it. ...",4,Cute Tutu,0,1
4,My little girl just loves to wear this tutu be...,5,Versatile,0,1


Here, an 80/20 split of the data will be utilized for training and testing, respectively.

In [5]:
# Split dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(data[['reviewText', 'overall', 'summary', 'helpful']], 
                                                    data['class'], test_size=0.2, random_state=42, shuffle=True)

We must convert text features (`reviewText` & `summary`) into numerical vectors suitable for ML training.